#### Game

In [9]:
import pygame
import sys
import random

pygame.init()
screen = pygame.display.set_mode((1280, 720))
clock = pygame.time.Clock()
pygame.display.set_caption("YOLOv5 Game")

game_font = pygame.font.Font("assets/MotleyForcesRegular-w1rZ3.ttf", 24)

# Variables
game_speed = 5


# Classes
class Player(pygame.sprite.Sprite):
    def __init__(self, x_pos, y_pos):
        super().__init__()

        self.running_sprites = []
        self.running_sprites.append(pygame.transform.scale(
            pygame.image.load("assets/Dino1.png"), (80, 100)))
        self.running_sprites.append(pygame.transform.scale(
            pygame.image.load("assets/Dino2.png"), (80, 100)))

        self.x = x_pos
        self.y = y_pos

        self.current_image = 0
        self.image = self.running_sprites[self.current_image]
        self.rect = self.image.get_rect(center=(self.x, self.y))  # Corrected line

    def update(self):
        self.animate()

    def animate(self):
        self.current_image += 0.05
        if self.current_image >= 2:
            self.current_image = 0

        self.image = self.running_sprites[int(self.current_image)]



# Surfaces
ground = pygame.image.load("assets/background.png")
ground = pygame.transform.scale(ground, (1280, 900))
ground_x = 0
ground_rect = ground.get_rect(center=(640, 400))

#Groups
player_group = pygame.sprite.GroupSingle()

# Objects
player = Player(50, 440)
player_group.add(player)
 

while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

    screen.fill("white")

    

    game_speed += 0.0025


    ground_x -=1

    screen.blit(ground, (ground_x, 0))
    screen.blit(ground, (ground_x + 1280, 0))

    player_group.update()
    player_group.draw(screen)

    if ground_x <= -1280:
        ground_x = 0

    pygame.display.update()
    clock.tick(120)






SystemExit: 

C:\Users\saniy\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
